Задание 1  
Для датафрейма log из материалов занятия создайте столбец source_type по правилам:

если источник traffic_source равен Yandex или Google, то в source_type ставится organic;  
для источников paid и email из России ставим ad;  
для источников paid и email не из России ставим other;  
все остальные варианты берём из traffic_source без изменений.

In [ ]:
import pandas as pd

log = pd.read_csv('/content/visit_log.csv', sep = ';')

"""Создание столбца source_type"""
log['source_type'] = ''

"""Применяем правила"""
log.loc[log['traffic_source'].isin(['yandex', 'google']), 'source_type'] = 'organic'
log.loc[(log['traffic_source'].isin(['paid', 'email'])) & (log['region'] == 'Russia'), 'source_type'] = 'ad'
log.loc[(log['traffic_source'].isin(['paid', 'email'])) & (log['region'] != 'Russia'), 'source_type'] = 'other'
log.loc[log['source_type'] == '', 'source_type'] = log['traffic_source']
print(log.head(20))

     timestamp    visit_id                                 url   region  \
0   1549980692  e3b0c44298  https://host.ru/3c19b4ef7371864fa3   Russia   
1   1549980704  6e340b9cff  https://host.ru/c8d9213a31839f9a3a   Russia   
2   1549980715  96a296d224  https://host.ru/b8b58337d272ee7b15   Russia   
3   1549980725  709e80c884  https://host.ru/b8b58337d272ee7b15   Russia   
4   1549980736  df3f619804  https://host.ru/b8b58337d272ee7b15   Russia   
5   1549980742  8855508aad  https://host.ru/df646c3676cc259fa0   Russia   
6   1549980742  b0f66adc83  https://host.ru/b8b58337d272ee7b15   Russia   
7   1549980754  837885c8f8  https://host.ru/108ce4b365afb7b88e   Russia   
8   1549980760  af5570f5a1  https://host.ru/3004a8273caeef2867    China   
9   1549980765  3e7077fd2f  https://host.ru/df646c3676cc259fa0   Russia   
10  1549980777  01d448afd9  https://host.ru/a5dda93e70318570c0    India   
11  1549980785  71b6c1d538  https://host.ru/c8d9213a31839f9a3a  Ukraine   
12  1549980789  15ec7bf0b

Задание 2

В файле URLs.txt содержатся URL страниц новостного сайта. Вам нужно отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри URL конструкцию: /, затем 8 цифр, затем дефис.  
Выполните действия:

Прочитайте содержимое файла с датафрейм.
Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствие с заданным шаблоном.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/URLs.txt', sep='\t')

"""Применение регулярного выражения для фильтрации строк"""
pattern = r'/[0-9]{8}-'
mask = df['url'].str.contains(pattern)

"""Создание нового датафрейма с отфильтрованными строками"""
news_df = df[mask]

print(df.head(10))

                                                 url
0                                            /world/
1                                           /latest/
2                                      /?updated=top
3  /politics/36188461-s-marta-zhizn-rossiyan-susc...
4  /world/36007585-tramp-pridumal-kak-reshit-ukra...
5  /science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6  /video/36001498-poyavilis-pervye-podrobnosti-g...
7  /world/36007585-tramp-pridumal-kak-reshit-ukra...
8                                          /science/
9                                            /sport/


Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значениями столбца timestamp для данного значения userId.

In [ ]:
import pandas as pd

df = pd.read_csv('/content/ratings.csv', sep = ',')

"""Группировка по userId и вычисление количества оценок, максимального и минимального значений timestamp"""
group_df = df.groupby('userId')['timestamp'].agg(['count', 'max', 'min'])

"""Фильтрация пользователей, у которых более 100 оценок"""
filter_df = group_df.query("count > 100")

"""Вычисляем разницу между максимальным и минимальным значением timestamp"""
filter_df['mean_time'] = filter_df['max'] - filter_df['min']

"""Рассчитываем среднее время жизни"""
mean_time = filter_df['mean_time'].mean()

"""Добавляем среднее время жизни в датафрейм"""
df['mean_time'] = mean_time

print(df.head(10))

   userId  movieId  rating   timestamp     mean_time
0       1       31     2.5  1260759144  4.008051e+07
1       1     1029     3.0  1260759179  4.008051e+07
2       1     1061     3.0  1260759182  4.008051e+07
3       1     1129     2.0  1260759185  4.008051e+07
4       1     1172     4.0  1260759205  4.008051e+07
5       1     1263     2.0  1260759151  4.008051e+07
6       1     1287     2.0  1260759187  4.008051e+07
7       1     1293     2.0  1260759148  4.008051e+07
8       1     1339     3.5  1260759125  4.008051e+07
9       1     1343     2.0  1260759131  4.008051e+07


<ipython-input-13-b4e67c9d5b91>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df['mean_time'] = filter_df['max'] - filter_df['min']


Задание 4

Дана статистика услуг перевозок клиентов компании по типам:  
rzd - железнодорожные перевозки  
auto - автомобильные перевозки  
air - воздушные перевозки  
client_base - адреса клиентов

Нужно сформировать две таблицы:  
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента;  
аналогичную таблицу по типам выручки с указанием адреса клиента.  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1',
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
"""первая таблица с тремя типами выручки для каждого клиента без адреса"""
table1 = pd.merge(rzd, auto, on='client_id', how='outer').merge(air, on='client_id', how='outer')
print(table1)

   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           NaN          NaN
1        112       2810.0           NaN          NaN
2        113      10283.0       57483.0          NaN
3        114       5774.0          83.0          NaN
4        115        981.0         912.0         81.0
5        116          NaN        4834.0          4.0
6        117          NaN          98.0         13.0
7        118          NaN           NaN        173.0


In [ ]:
"""вторая таблица по типам выручки для каждого клиента с адресом"""
table2 = pd.merge(table1, client_base, on ='client_id', how='outer')
print(table2)

   client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           NaN          NaN  Комсомольская 4
1        112       2810.0           NaN          NaN   Энтузиастов 8а
2        113      10283.0       57483.0          NaN  Левобережная 1а
3        114       5774.0          83.0          NaN          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          NaN        4834.0          4.0    Строителей 18
6        117          NaN          98.0         13.0  Панфиловская 33
7        118          NaN           NaN        173.0     Мастеркова 4
